# REQUEST FILE GENERATION

In [5]:
import random
import networkx as nx

def generate_requests(graph, num_requests, demand_type):
    result = []
    node_request = len(graph.nodes) 
    
    for _ in range(num_requests):
        source = random.choice(list(graph.nodes))
        destination = random.choice(list(graph.nodes))
        while destination == source:
            destination = random.choice(list(graph.nodes))
        if demand_type == 'variable':
            demand = 12.5 * random.choice([1, 2, 3, 4, 5, 6])
        else:
            demand = demand
        
        new_req = {
            "source": source,
            "destination": destination,
            "demand": demand
        }
        
        result.append(new_req)
        
    return result

if __name__ == "__main__":
    file_path = input("Enter file name : ")
    
    try:
        graph = nx.read_gpickle(file_path+".gpickle")
        num_requests = int(input("Enter the number of requests: "))
        demand_type = input("Enter demand type (40 or 80 for fixed, or variable): ").lower()
        if demand_type == '40':
            demand = 40
            
            
        elif demand_type == '80':
            demand = 80
        elif demand_type == 'variable':
            demand = "variable"
        else:
            print("Invalid demand type.")
            exit()
        requests = generate_requests(graph, num_requests, demand_type)
        for req in requests:
            print(req)
    
    except FileNotFoundError:
        print("The specified gpickle file was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


Enter file name : network_1
Enter the number of requests: 200
Enter demand type (40 or 80 for fixed, or variable): variable
{'source': 21, 'destination': 12, 'demand': 37.5}
{'source': 16, 'destination': 13, 'demand': 12.5}
{'source': 13, 'destination': 11, 'demand': 37.5}
{'source': 19, 'destination': 21, 'demand': 25.0}
{'source': 15, 'destination': 12, 'demand': 50.0}
{'source': 10, 'destination': 1, 'demand': 12.5}
{'source': 22, 'destination': 18, 'demand': 62.5}
{'source': 5, 'destination': 10, 'demand': 12.5}
{'source': 21, 'destination': 11, 'demand': 75.0}
{'source': 16, 'destination': 23, 'demand': 50.0}
{'source': 7, 'destination': 22, 'demand': 75.0}
{'source': 10, 'destination': 1, 'demand': 25.0}
{'source': 13, 'destination': 9, 'demand': 12.5}
{'source': 17, 'destination': 2, 'demand': 25.0}
{'source': 4, 'destination': 20, 'demand': 75.0}
{'source': 5, 'destination': 20, 'demand': 12.5}
{'source': 0, 'destination': 13, 'demand': 25.0}
{'source': 16, 'destination': 14, '

# DATA GENERATION

In [6]:
import networkx as nx

def load_network_graph(filename):
    return nx.read_gpickle(filename)

def sparse_regeneration_algorithm(network_graph, requests, distance_threshold , Frequency_Slot_Capacity ):
    results = [] 
    total_regen_primary=0
    total_regen_backup=0
    Total_Frequency_Slot=0
    Guard_Frequency_Slot=2
    for request in requests:
        source = request['source']
        destination = request['destination']
        demand = request['demand']  
        
        
        try:   
            primary_path = nx.dijkstra_path(graph, source=source, target=destination, weight='weight')
            total_distance_primary = sum(graph[primary_path[i]][primary_path[i+1]]['weight'] for i in range(len(primary_path) - 1))
            
            graph_backup = graph.copy()
            for i in range(len(primary_path) - 1):
                graph_backup.remove_edge(primary_path[i], primary_path[i+1])
            try:
                backup_path = nx.dijkstra_path(graph_backup, source=source, target=destination, weight='weight')
                total_distance_backup = sum(graph_backup[backup_path[i]][backup_path[i+1]]['weight'] for i in range(len(backup_path) - 1))
            except nx.NetworkXNoPath:
                backup_path = None
                total_distance_backup = None
            
            regenerations_primary = 0
            regenerations_backup = 0
            current_distance_primary = 0
            current_distance_backup = 0
            regeneration_nodes_primary = []
            regeneration_nodes_backup = []
            path_segments_primary = []
            path_segments_backup = []
            segment_start_primary = source
            segment_start_backup = source
            
#             FOR PRIMARY PATH
            
            for i in range(1, len(primary_path)):
                link_distance = graph[primary_path[i-1]][primary_path[i]]['weight']
                current_distance_primary += link_distance 
                N = (demand//Frequency_Slot_Capacity)+Guard_Frequency_Slot
                Total_Frequency_Slot+=N
                
                
                if current_distance_primary > distance_threshold:
                    regenerations_primary += 1
                    total_regen_primary+=1
                    regeneration_nodes_primary.append(primary_path[i-1])
                    
                    if segment_start_primary != primary_path[i-1]:
                        path_segments_primary.append((segment_start_primary, primary_path[i-1]))
                    segment_start_primary = primary_path[i-1]
                        
                    current_distance_primary -= distance_threshold
            
            path_segments_primary.append((segment_start_primary, destination))
            
#             FOR BACKUP PATH

            if backup_path:
                for i in range(1, len(backup_path)):
                    link_distance = graph_backup[backup_path[i-1]][backup_path[i]]['weight']
                    current_distance_backup += link_distance
                    
                    if current_distance_backup > distance_threshold:
                        regenerations_backup += 1
                        regeneration_nodes_backup.append(backup_path[i-1])
                        
                        if segment_start_backup != backup_path[i-1]:
                            path_segments_backup.append((segment_start_backup, backup_path[i-1]))
                        segment_start_backup = backup_path[i-1]
                        
                        current_distance_backup -= distance_threshold
                
                path_segments_backup.append((segment_start_backup, destination))
            
            results.append({
                'source': source,
                'destination': destination,
                'demand':demand,
                
                'primary_path': primary_path,
                'total_distance_primary': total_distance_primary,
                'path_segments_primary': path_segments_primary,
                'regenerations_primary': regenerations_primary,
                'regeneration_nodes': regeneration_nodes_primary,
                
                'backup_path':backup_path,
                'total_distance_backup': total_distance_backup,
                'path_segments_backup': path_segments_backup,
                'regenerations_backup': regenerations_backup,
                'regeneration_nodes_backup': regeneration_nodes_backup,
                
                'Total_Regenerations_Primary' : total_regen_primary,
                'Total_Frequency_Slot' : Total_Frequency_Slot,
                'Spectrum_Allocated':N,
                
            })
        except nx.NetworkXNoPath:
            print(f"No path found between {source} and {destination}.")
            results.append({
                'source': source,
                'destination': destination,
                'demand':None,
                'primary_path': None,
                'total_distance_primary': None,
                'backup_path':None,
                'total_distance_backup': None,
                'path_segments_primary': None,
                'regenerations_primary': None,
                'regeneration_nodes': None,
                'Total_Regenerations_Primary' : None,
                'Total_Frequency_slot' : None,
                'Spectrum_Allocated':None,
            })
    return results

network_graph=graph

distance_threshold = 2000  
Frequency_Slot_Capacity = 25

results = sparse_regeneration_algorithm(network_graph, requests, distance_threshold , Frequency_Slot_Capacity)

for result in results:
    print("\nRequest from {} to {}".format(result['source'], result['destination']))
    print("  Demand:",result["demand"])
    
    print("  Path (primary):", result['primary_path'])
    print("  Total Distance (primary):", result['total_distance_primary'])
    print("  Regenerations Required:", result['regenerations_primary'])
    print("  Regeneration_at_nodes:",result["regeneration_nodes"])
    print("  Spectrum Allocated:",result["Spectrum_Allocated"])
    print("  Segment (primary):",result["path_segments_primary"] )
    
    print("\n  Backup Path:", result['backup_path'])
    print("  Total Distance (Backup):", result['total_distance_backup'])
    print("  Regenerations Required (backup):", result['regenerations_backup'])
    print("  Regeneration_at_nodes (backup):", result["regeneration_nodes_backup"])
    print("  Segment (backup):", result["path_segments_backup"])
    
    
print("\n\nTotal_Regenerations:" ,result["Total_Regenerations_Primary"])
print("Total_Frequency_Slot:" ,result["Total_Frequency_Slot"])


Request from 21 to 12
  Demand: 37.5
  Path (primary): [21, 16, 12]
  Total Distance (primary): 1950
  Regenerations Required: 0
  Regeneration_at_nodes: []
  Spectrum Allocated: 3.0
  Segment (primary): [(21, 12)]

  Backup Path: [21, 15, 11, 12]
  Total Distance (Backup): 2700
  Regenerations Required (backup): 1
  Regeneration_at_nodes (backup): [11]
  Segment (backup): [(21, 11), (11, 12)]

Request from 16 to 13
  Demand: 12.5
  Path (primary): [16, 12, 13]
  Total Distance (primary): 1750
  Regenerations Required: 0
  Regeneration_at_nodes: []
  Spectrum Allocated: 2.0
  Segment (primary): [(16, 13)]

  Backup Path: [16, 17, 13]
  Total Distance (Backup): 2000
  Regenerations Required (backup): 0
  Regeneration_at_nodes (backup): []
  Segment (backup): [(16, 13)]

Request from 13 to 11
  Demand: 37.5
  Path (primary): [13, 12, 11]
  Total Distance (primary): 1550
  Regenerations Required: 0
  Regeneration_at_nodes: []
  Spectrum Allocated: 3.0
  Segment (primary): [(13, 11)]

  B

# SPECTRUM ALLOCATION

In [3]:
import numpy as np

total_segments=0
segment_names = []

for result in results:
    if result["path_segments_primary"]:
        for segment in result["path_segments_primary"]:
            total_segments += 1
            segment_names.append(f"segment_primary_{segment[0]}_{segment[1]}")

    if result["path_segments_backup"]:
        for segment in result["path_segments_backup"]:
            total_segments += 1
            segment_names.append(f"segment_backup_{segment[0]}_{segment[1]}")

array = np.zeros((total_segments, 100))

print("\nTotal Segments:", total_segments)
print("Segment Names:")
for i, name in enumerate(segment_names):
    print(f"Row {i}: {name}")

print("\n2D Array initialized to zeros:")
print(array)
print(array.shape)



Total Segments: 3
Segment Names:
Row 0: segment_primary_16_14
Row 1: segment_backup_16_19
Row 2: segment_backup_19_14

2D Array initialized to zeros:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 

# SHOWING SPECTRUM ALLOCATION

In [46]:
import numpy as np

spectrum_slot=100

# Initialize a list to track the starting position of each segment's allocation
segment_start_positions = {}

# Collect all unique segments from the results (including duplicates)
segment_names = []
for result in results:
    for segment in result['path_segments_primary']:
        segment_names.append(f"segment_primary_{segment[0]}_{segment[1]}")
    for segment in result.get('path_segments_backup', []):
        segment_names.append(f"segment_backup_{segment[0]}_{segment[1]}")

# Initialize the 2D array for spectrum allocation
total_segments = len(segment_names)
spectrum_array = np.zeros((total_segments, spectrum_slot))  # 100 is the maximum number of spectrum slots
allocated_spectrum = [0] * total_segments  # List to track allocated spectrum count for each segment

# Function to allocate spectrum to segments
for result in results:
    for segment in result['path_segments_primary']:
        segment_name = f"segment_primary_{segment[0]}_{segment[1]}"
        
        # Find the row index for this segment in segment_names
        if segment_name in segment_names:
            row_index = segment_names.index(segment_name)
            N = int(result["Spectrum_Allocated"])  # Spectrum to allocate
            
            # Check if spectrum has already been allocated for this segment
            if allocated_spectrum[row_index] == 0:
                # First allocation, allocate from the start
                spectrum_array[row_index, allocated_spectrum[row_index]:allocated_spectrum[row_index] + N] = 1
                allocated_spectrum[row_index] += N
                # Track the starting position
                segment_start_positions[segment_name] = allocated_spectrum[row_index] - N
            else:
                # Spectrum already allocated, continue from the first zero
                start_index = np.argmax(spectrum_array[row_index] == 0)
                if start_index + N <= spectrum_slot:
                    spectrum_array[row_index, start_index:start_index + N] = 1
                    allocated_spectrum[row_index] = start_index + N
                else:
                    # Request blocked for this segment
                    print(f"Request blocked for {segment_name}: Not enough spectrum available.")

    # Allocate spectrum for backup path if available
    for segment in result.get('path_segments_backup', []):
        segment_name = f"segment_backup_{segment[0]}_{segment[1]}"
        
        if segment_name in segment_names:
            row_index = segment_names.index(segment_name)
            N = int(result["Spectrum_Allocated"])  # Spectrum to allocate
            
            if allocated_spectrum[row_index] == 0:
                # First allocation, allocate from the start
                spectrum_array[row_index, allocated_spectrum[row_index]:allocated_spectrum[row_index] + N] = 1
                allocated_spectrum[row_index] += N
                segment_start_positions[segment_name] = allocated_spectrum[row_index] - N
            else:
                # Spectrum already allocated, continue from the first zero
                start_index = np.argmax(spectrum_array[row_index] == 0)
                if start_index + N <= spectrum_slot:
                    spectrum_array[row_index, start_index:start_index + N] = 1
                    allocated_spectrum[row_index] = start_index + N
                else:
                    # Request blocked for this segment
                    print(f"Request blocked for {segment_name}: Not enough spectrum available.")

# Display the spectrum allocation and track the start positions
print("\nSpectrum Allocation for each segment:")
for i, name in enumerate(segment_names):
    print(f"{name}: {spectrum_array[i]} (Start Position: {segment_start_positions.get(name, 'N/A')})")

# Final allocated spectrum count
print("\nTotal Spectrum Allocated for each segment:")
for i, name in enumerate(segment_names):
    print(f"{name}: {allocated_spectrum[i]}")



Spectrum Allocation for each segment:
segment_primary_16_11: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.] (Start Position: 0)
segment_primary_11_8: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.] (Start Position: 0)
segment_backup_16_12: [1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0

# CREATING LINK DISJOINT PATHS GROUP

In [4]:
disjoint_groups = []

def are_paths_link_disjoint(path1, path2, graph):
    for i in range(len(path1) - 1):
        edge1 = (path1[i], path1[i + 1])
        edge1_reverse = (path1[i + 1], path1[i])
        for j in range(len(path2) - 1):
            edge2 = (path2[j], path2[j + 1])
            edge2_reverse = (path2[j + 1], path2[j])
            if edge1 == edge2 or edge1 == edge2_reverse:
                return False
    return True

def is_path_already_in_groups(path, groups):
    for group in groups:
        for existing_path in group:
            if path == existing_path:
                return True
    return False

for result in results:
    primary_path = result['primary_path']
    if primary_path is not None:
        if is_path_already_in_groups(primary_path, disjoint_groups):
            continue
        
        placed = False
        for group in disjoint_groups:
            is_disjoint = True
            for existing_path in group:
                if not are_paths_link_disjoint(primary_path, existing_path, network_graph):
                    is_disjoint = False
                    break
            if is_disjoint:
                group.append(primary_path)
                placed = True
                break
        if not placed:
            disjoint_groups.append([primary_path])

print("\nDisjoint Groups of Paths:")
for idx, group in enumerate(disjoint_groups, 1):
    print(f"\nGroup {idx}:\n")
    for path in group:
        print(path)



Disjoint Groups of Paths:

Group 1:

[16, 15, 14]
